In [20]:
import os

def rename_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        # Full path to the file
        file_path = os.path.join(folder_path, filename)
        
        # Ensure it is a file
        if os.path.isfile(file_path):
            # Remove the .txt extension
            name, ext = os.path.splitext(filename)
            if ext == '.txt':
                # Check if the prefix contains _ktr and rename accordingly
                if '_ktr' in name:
                    new_name = name.replace('_ktr', '') + '.ktr'
                elif '_kjb' in name:
                    new_name = name.replace('_kjb', '') + '.kjb'
                else:
                    new_name = name
                
                # Full new path
                new_file_path = os.path.join(folder_path, new_name)
                
                # Rename the file
                os.rename(file_path, new_file_path)
                print(f'Renamed: {file_path} -> {new_file_path}')

# Example usage
folder_path = 'C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy'
rename_files_in_folder(folder_path)

Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\BMCS001_JOB_CONTROL_kjb.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\BMCS001_JOB_CONTROL.kjb
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\code conersion_ktr.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\code conersion.ktr
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\get_business_date_in_range_ktr.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\get_business_date_in_range.ktr
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\job_ad_ldap_sync_kjb.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\job_ad_ldap_sync.kjb
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\JOB_KYC_MAIN_kjb.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\JOB_KYC_MAIN.kjb
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\JOB_MV_GRU_PREPARE_TEMP_FILE_kjb.txt -> C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\JOB_MV_GRU_PREPARE_TEMP_FILE.kjb
Renamed: C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\JOB_OB_

In [21]:
import pandas as pd
import json
import openai
import os
import shutil
import xml.etree.ElementTree as ET
import requests
import logging
from logging.handlers import RotatingFileHandler

# Configure logging with a rotating log file handler
log_filename = 'app.log'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
handler = RotatingFileHandler(log_filename, maxBytes=10*1024*1024, backupCount=1)
handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger = logging.getLogger(__name__)
logger.addHandler(handler)

file_path = None
folder_name = None

def get_elements_from_file(xml_file_path, element_name):
    root = ET.parse(xml_file_path).getroot()
    step_elements = root.findall(f'.//{element_name}')
    return [ET.tostring(step, encoding='unicode') for step in step_elements] if step_elements else []

def save_elements_to_file(input_filename, output_filename):
    xml_file_path = f'{input_filename}'
    output_text_file_path = f'{output_filename}s.txt' if output_filename == 'step' else 'hop_order.txt'
    step_data_strings = get_elements_from_file(xml_file_path, output_filename)

    logger.info(f"{output_filename} elements extracted")

    with open(output_text_file_path, 'w', encoding='utf-8') as output_file:
        output_file.writelines(f"{step_data_string}\n" for step_data_string in step_data_strings)

    logger.info(f"The output has been saved to '{output_text_file_path}'")
    print(f"The output has been saved to '{output_text_file_path}'")

def openai_sequence_steps(hop_order_file_path, steps_file_path):
    # Read the configuration file for OpenAI credentials
    with open('config_file.json') as f:
        json_data = json.load(f)

    # Set up OpenAI configuration with the loaded credentials
    openai.api_type = json_data["openai_api_type"]
    openai.api_base = json_data["openai_api_base"]
    openai.api_version = json_data["openai_api_version"]
    openai.api_key = json_data['openai_api_key']
  
    # Read the contents of the steps file
    with open(hop_order_file_path, 'r', encoding='utf-8') as file:
        hop_order = file.read()
        
    # Read the contents of the steps file
    with open(steps_file_path, 'r', encoding='utf-8') as file:
        steps = file.read()
  
    try:
        # Make the API call to OpenAI with the hop info and the contents of steps.txt
        response = openai.ChatCompletion.create(
            engine="gpt-4-32k",
            temperature=0.1,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in converting Pentaho code to Spark code.\n\nYou will receive a hop order file and a steps file. Analyze the hop order to determine the sequence, then convert the corresponding steps into Spark code following that sequence.\n\nReturn the Spark code for the entire flow step by step as per the steps file.\n\nGive sequence number to each step."
                },
                {   "role": "user", 
                    "content": f"Here is the Pentaho hop order file: {hop_order} and the steps file {steps}"
                }
            ],
        )
      
        # Extract the output from the response
        output = response["choices"][0]["message"]["content"]
        return output
    except Exception as e:
        print(e)
        logger.error(f"{e}")
        raise

def copy_specific_files(source_folder, target_folder, file_names):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    for file_name in file_names:
        source_path = os.path.join(source_folder, file_name)
        target_path = os.path.join(target_folder, file_name)
        if os.path.exists(source_path):
            shutil.copy(source_path, target_path)
            print(f"Copied '{file_name}' to '{target_folder}'.")
        else:
            print(f"File '{file_name}' not found in '{source_folder}'.")




In [22]:

def process_ktr(file_path, folder_name):
        input_filename = file_path
        folder_name = os.path.join('spark_code', folder_name)
        hop_order_file_path = "hop_order.txt"
        steps_file_path = "steps.txt"
        output_file_path = "pyspark_code.txt"

        save_elements_to_file(input_filename, 'order')
        save_elements_to_file(input_filename, 'step')

        code_output = openai_sequence_steps(hop_order_file_path, steps_file_path)
        print(code_output)

        with open(output_file_path, 'a', encoding='utf-8') as output_file:
            output_file.write(code_output)

        source_folder = ""
        target_folder = folder_name
        file_names = ["hop_order.txt", "steps.txt", "pyspark_code.txt"]
        copy_specific_files(source_folder, target_folder, file_names)
        for file_name in file_names:
            file_path = os.path.join(source_folder, file_name)
            os.remove(file_path)
        logger.info(f"The converted code is saved in {folder_name} folder")


In [23]:
def process_directory(directory_path):
    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Process the file based on its extension
            if file_path.endswith('.ktr'):
                print('file_path is', file_path)
                file_name = os.path.basename(file_path)
                folder_name = os.path.splitext(file_name)[0]
                process_ktr(file_path, folder_name)

# Start the processing with the main directory
main_directory_path = r'C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy'  # Replace with the path to your directory
process_directory(main_directory_path)

2024-06-13 11:13:30,488 - INFO - order elements extracted
2024-06-13 11:13:30,492 - INFO - The output has been saved to 'hop_order.txt'
2024-06-13 11:13:30,555 - INFO - step elements extracted
2024-06-13 11:13:30,560 - INFO - The output has been saved to 'steps.txt'


file_path is C:\Multiple_KTR_Folder\KTR_Folder\Medium&Easy\code conersion.ktr
The output has been saved to 'hop_order.txt'
The output has been saved to 'steps.txt'


2024-06-13 11:14:51,337 - INFO - The converted code is saved in spark_code\code conersion folder
2024-06-13 11:14:51,342 - INFO - order elements extracted
2024-06-13 11:14:51,346 - INFO - The output has been saved to 'hop_order.txt'
2024-06-13 11:14:51,353 - INFO - step elements extracted
2024-06-13 11:14:51,357 - INFO - The output has been saved to 'steps.txt'


Based on the hop order and steps file, the sequence of steps and corresponding Spark code would be as follows:

1. BetType 2
```python
# Step 1: BetType 2
df_BetType2 = spark.read.format('com.crealytics.spark.excel') \
    .option('useHeader', 'true') \
    .option('inferSchema', 'true') \
    .load('${input_path}\BetType.xls')
```

2. Select values
```python
# Step 2: Select values
df_SelectValues = df_BetType2.select('BETTYPE', df_BetType2['Bet Type Description '].alias('BetTypeDescription'))
```

3. BetType
```python
# Step 3: BetType
df_SelectValues.write.parquet('maprfs:///raw/common/MasterDataCodeDescription/BetType', mode='overwrite')
```

4. Microsoft Excel Input 2 2
```python
# Step 4: Microsoft Excel Input 2 2
df_MicrosoftExcelInput22 = spark.read.format('com.crealytics.spark.excel') \
    .option('useHeader', 'true') \
    .option('inferSchema', 'true') \
    .load('${input_path}\Commingling.xls')
```

5. Replace in string
```python
# Step 5: Replace in string
from pyspark.s

2024-06-13 11:15:35,283 - INFO - The converted code is saved in spark_code\get_business_date_in_range folder
2024-06-13 11:15:35,290 - INFO - order elements extracted
2024-06-13 11:15:35,293 - INFO - The output has been saved to 'hop_order.txt'
2024-06-13 11:15:35,301 - INFO - step elements extracted
2024-06-13 11:15:35,306 - INFO - The output has been saved to 'steps.txt'


...

Based on the hop order and steps file, the sequence of steps and their corresponding Spark code would be as follows:

1. get var
```python
# No equivalent Spark code as this is a variable retrieval step in Pentaho
```

2. tablename
```python
# No equivalent Spark code as this is a script execution step in Pentaho
```

3. slct 3
```python
# No equivalent Spark code as this is a select values step in Pentaho
```

4. Table exists
```python
# Spark doesn't have a built-in function to check if a table exists. However, you can use the following code:
spark.sql("SHOW TABLES LIKE 'tablename'").count() > 0
```

5. tbl_not_exist1
```python
# No equivalent Spark code as this is a filter rows step in Pentaho
```

6. table_exist
```python
# No equivalent Spark code as this is a select values step in Pentaho
```

7. metadata
```python
# No equivalent Spark code as this is a table input step in Pentaho
```

8. data_type_change
```python
# No equivalent Spark code as this is a replace string step in P

2024-06-13 11:40:30,923 - INFO - The converted code is saved in spark_code\tr_views_priv_remove folder
2024-06-13 11:40:30,935 - INFO - order elements extracted
2024-06-13 11:40:30,938 - INFO - The output has been saved to 'hop_order.txt'
2024-06-13 11:40:30,945 - INFO - step elements extracted
2024-06-13 11:40:30,950 - INFO - The output has been saved to 'steps.txt'


Based on the hop order and the steps file, the sequence of steps is as follows:

1. Get Variables
2. user
3. View
4. Iselect
5. Output

Now, let's convert these steps into Spark code:

```python
# Step 1: Get Variables
pviewsave = spark.read.format("csv").option("header", "true").load("${pviewsave}")
pviewuser = spark.read.format("csv").option("header", "true").load("${pviewuser}")
pviewrole = spark.read.format("csv").option("header", "true").load("${pviewrole}")

# Step 2: user
user = pviewuser.withColumn("portaluser", F.split(F.col("portaluser"), ","))

# Step 3: View
view = pviewsave.withColumn("pagelist", F.split(F.col("pagelistt"), ","))

# Step 4: Iselect
iselect = view.select("pagelist", "portaluser_1")

# Step 5: Output
output = iselect.write.format("jdbc").option("url", "jdbc:postgresql:ABCD").option("dbtable", "dim_viewprivilige").option("user", "username").option("password", "password").save()
```

Please replace `"username"` and `"password"` with your actual database userna

2024-06-13 11:41:30,025 - INFO - The converted code is saved in spark_code\tr_views_priv_viewlist_updt folder


Based on the hop order and the steps file, the sequence of steps and their corresponding Spark code would be as follows:

1. Get Variables
```scala
val pviewsave = spark.sqlContext.getConf("pviewsave")
val pviewuser = spark.sqlContext.getConf("pviewuser")
val pviewrole = spark.sqlContext.getConf("pviewrole")
```

2. Filter rows
```scala
val filteredDF = df.filter(col("pagelistt").contains(","))
```

3. DView
```scala
val splitDF = filteredDF.withColumn("pagelist", explode(split(col("pagelistt"), ",")))
```

4. Dselect
```scala
val selectedDF = splitDF.select("pagelist", "portaluser", "pviewrole", col("pviewrole").as("pviewrole_1"))
```

5. delete priviliges
```scala
spark.sql("delete from dim_viewprivilige where view_id in (select view_id from dim_view where business_name = ? and category_id<>6) and user_id in(select a.user_id from dim_user a join dim_viewprivilige b on a.user_id=b.user_id where status=1) and role_id in (select role_id from dim_role where (role_name = ? or 'ALL'=?) and